In [1]:
import pandas as pd

In [2]:
# import the bitcoin and ethereum google trend dataset from https://trends.google.com/trends/explore taken from 12/18/2016 
# to 11/22/2021. Also imports bitcoin and ethereum market price from https://www.coindesk.com/
btc_trends = pd.read_csv("bitcoin_google_trend.csv")
eth_trends = pd.read_csv("ethereum_google_trend.csv")
btc_price = pd.read_csv("bitcoin_price.csv")
eth_price = pd.read_csv("ethereum_price.csv")
# Rename and drop unnecessary columns
btc_price = btc_price.rename(columns = {'Date' : 'Week',
                                        'Closing Price (USD)' : 'BTC_Close'})
btc_trends = btc_trends.rename(columns = {'bitcoin: (United States)' : 'BTC_Searches'})
btc_price.drop(btc_price.columns[[0,3,4,5]], axis=1, inplace=True)
btc_price["BTC_Percent_Change"] = ((btc_price["BTC_Close"] - btc_price["BTC_Close"].shift(+1)) / btc_price["BTC_Close"] * 100)
eth_price = eth_price.rename(columns = {'Date' : 'Week', 
                                        'Closing Price (USD)' : 'ETH_Close'})
eth_trends = eth_trends.rename(columns = {'ethereum: (United States)' : 'ETH_Searches'})
eth_price.drop(eth_price.columns[[0,3,4,5]], axis=1, inplace=True)
btc_price

,Week,BTC_Close,BTC_Percent_Change
0,2014-11-02,331.600830,NaN
1,2014-11-03,324.718330,-2.119529
2,2014-11-04,332.456660,2.327621
3,2014-11-05,336.585000,1.226537
4,2014-11-06,346.775000,2.938505
...,...,...,...
2569,2021-11-18,56936.516170,-5.468203
2570,2021-11-19,57896.166603,1.657537
2571,2021-11-20,59756.024151,3.112418
2572,2021-11-21,59023.259799,-1.241484


In [3]:
# Merge the bitcoin dataframes together
bitcoin = btc_trends.merge(btc_price)
bitcoin.tail()

,Week,BTC_Searches,BTC_Close,BTC_Percent_Change
252,2021-10-17,27,60965.465434,0.361648
253,2021-10-24,24,60936.150851,-0.308842
254,2021-10-31,19,61374.278373,0.051613
255,2021-11-07,21,62924.521859,2.311576
256,2021-11-14,23,64657.424112,0.404492


In [4]:
# Merge the ethereum dataframes together
ethereum = eth_trends.merge(eth_price)
ethereum.head()

,Week,ETH_Searches,ETH_Close
0,2016-12-18,1,7.735086
1,2016-12-25,1,7.260191
2,2017-01-01,2,8.212046
3,2017-01-08,1,10.266545
4,2017-01-15,1,9.663497


In [5]:
# Merge together all the data
merged = bitcoin.merge(ethereum, on = "Week")
merged.head()
print(merged.dtypes)

Week                   object
BTC_Searches            int64
BTC_Close             float64
BTC_Percent_Change    float64
ETH_Searches           object
ETH_Close             float64
dtype: object


In [6]:
# Create a SQL engine
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [7]:
# Add dataframe to SQL engine
merged.to_sql('BTC_ETH', con=engine)

In [8]:
# Look at all the data
engine.execute("SELECT * FROM BTC_ETH").fetchall()

[(0, '2016-12-18', 3, 791.0, 0.22171302149177852, '1', 7.735086),
 (1, '2016-12-25', 4, 905.45625, 3.9013480772814737, '1', 7.260191),
 (2, '2017-01-01', 6, 1009.97375, 4.519795687759206, '2', 8.212046),
 (3, '2017-01-08', 4, 889.33875, -3.8438671428631603, '1', 10.266545),
 (4, '2017-01-15', 3, 830.74, 1.1488251438476593, '1', 9.663497),
 (5, '2017-01-22', 3, 920.365, -1.5659548114063466, '1', 10.684885),
 (6, '2017-01-29', 3, 916.91125, -0.42724963839194013, '1', 10.40558),
 (7, '2017-02-05', 3, 1019.3125, -0.9897592740204711, '1', 11.217052),
 (8, '2017-02-12', 3, 1004.85375, 0.013932375731292874, '1', 11.280685),
 (9, '2017-02-19', 4, 1046.66249, -0.5905685986702424, '1', 12.830235),
 (10, '2017-02-26', 4, 1189.366, 4.16339461528243, '4', 14.706219),
 (11, '2017-03-05', 5, 1273.24699, 0.98794578732915, '2', 19.109602),
 (12, '2017-03-12', 4, 1235.553, 3.982184495525485, '7', 24.637882),
 (13, '2017-03-19', 4, 1036.226, 3.472216485592921, '5', 42.356981),
 (14, '2017-03-26', 3, 968.

In [9]:
#Finding the top search trend time for ETH
engine.execute("SELECT Week, ETH_Close, ETH_Searches FROM BTC_ETH WHERE ETH_Searches == 100").fetchall()

[('2021-05-02', 2941.1423331647, '100')]

In [10]:
#Finding the top search trend time for BTC
engine.execute("SELECT Week, BTC_Close, BTC_Searches FROM BTC_ETH WHERE BTC_Searches == 100").fetchall()

[('2017-12-17', 18640.2625, 100)]

In [11]:
# Do more searches result in higher or lower changes in stock price?
engine.execute("SELECT Week, BTC_Close, BTC_Searches, BTC_Percent_Change FROM BTC_ETH WHERE BTC_Searches > 40").fetchall()

[('2017-11-26', 9573.9675, 62, 7.548202978545731),
 ('2017-12-03', 11382.20875, 91, 3.5903839841278526),
 ('2017-12-10', 16299.2975, 83, 19.306161814642632),
 ('2017-12-17', 18640.2625, 100, -2.8256911081590075),
 ('2017-12-24', 13387.34875, 55, -5.347362001008608),
 ('2017-12-31', 13439.4175, 41, 2.807208720169599),
 ('2018-01-14', 13519.92625, 51, -5.158580284415374),
 ('2018-02-04', 8136.39, 43, -12.97448868601431),
 ('2021-01-03', 33002.5364270397, 50, 2.5706177633974487),
 ('2021-01-10', 38709.7653748751, 52, -4.001983992360734),
 ('2021-02-07', 38461.6814033048, 46, -4.786890022380313),
 ('2021-02-14', 49151.167576322, 43, 4.366075178363197),
 ('2021-02-21', 57128.6426064698, 45, 4.073252673802144),
 ('2021-05-16', 45604.6157536131, 53, -5.001707532033716)]